<a href="https://colab.research.google.com/github/rahiakela/transformers-research-and-practice/blob/main/attention-and-transformers-mechanism/neural-machine-translation/nmt_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Neural Machine Translation With Attention Mechanism

Today, let’s join me in the journey of creating a neural machine translation model with attention mechanism by using the hottest-on-the-news Tensorflow 2.0.

With that being said, our objective is pretty simple: we will use a very simple dataset (with only 20 examples) and we will try to overfit the training data with the renown Seq2Seq model. For the attention mechanism, we’re gonna use Luong attention, which I personally prefer over Bahdanau’s.

Without talking too much about theories today, let’s jump right into the implementation. As usual, we will go through the steps below:

* Data Preparation
* Seq2Seq without Attention
* Seq2Seq with Luong Attention

##Setup

In [4]:
import tensorflow as tf
import numpy as np
import unicodedata
import re

##Data Preparation

Let’s talk about the data. We’re gonna use 20 English – French pairs (which I extracted from the original dataset).

In [1]:
raw_data = (
    ('What a ridiculous concept!', 'Quel concept ridicule !'),
    ('Your idea is not entirely crazy.', "Votre idée n'est pas complètement folle."),
    ("A man's worth lies in what he is.", "La valeur d'un homme réside dans ce qu'il est."),
    ('What he did is very wrong.', "Ce qu'il a fait est très mal."),
    ("All three of you need to do that.", "Vous avez besoin de faire cela, tous les trois."),
    ("Are you giving me another chance?", "Me donnez-vous une autre chance ?"),
    ("Both Tom and Mary work as models.", "Tom et Mary travaillent tous les deux comme mannequins."),
    ("Can I have a few minutes, please?", "Puis-je avoir quelques minutes, je vous prie ?"),
    ("Could you close the door, please?", "Pourriez-vous fermer la porte, s'il vous plaît ?"),
    ("Did you plant pumpkins this year?", "Cette année, avez-vous planté des citrouilles ?"),
    ("Do you ever study in the library?", "Est-ce que vous étudiez à la bibliothèque des fois ?"),
    ("Don't be deceived by appearances.", "Ne vous laissez pas abuser par les apparences."),
    ("Excuse me. Can you speak English?", "Je vous prie de m'excuser ! Savez-vous parler anglais ?"),
    ("Few people know the true meaning.", "Peu de gens savent ce que cela veut réellement dire."),
    ("Germany produced many scientists.", "L'Allemagne a produit beaucoup de scientifiques."),
    ("Guess whose birthday it is today.", "Devine de qui c'est l'anniversaire, aujourd'hui !"),
    ("He acted like he owned the place.", "Il s'est comporté comme s'il possédait l'endroit."),
    ("Honesty will pay in the long run.", "L'honnêteté paye à la longue."),
    ("How do we know this isn't a trap?", "Comment savez-vous qu'il ne s'agit pas d'un piège ?"),
    ("I can't believe you're giving up.", "Je n'arrive pas à croire que vous abandonniez."),
)

Next, we will need to clean up the raw data a little bit. This kind of task usually involves normalizing strings, filtering unwanted tokens, adding space before punctuation, etc.

In [2]:
def unicode_to_ascii(sent):
  return "".join(char for char in unicodedata.normalize("NFD", sent) if unicodedata.category(char) != "Mn")

def normalize_string(sent):
  sent = unicode_to_ascii(sent)
  sent = re.sub(r"([!.?])", r"\1", sent)
  sent = re.sub(r"[^a-zA-Z.!?]+", r" ", sent)
  sent = re.sub(r"\s+", r" ", sent)
  return sent

We will now split the data into two separate lists, each containing its own sentences. 

Then we will apply the functions above and add two special tokens: `<start> and <end>`:

In [5]:
raw_data_en, raw_data_fr = list(zip(*raw_data))
raw_data_en, raw_data_fr = list(raw_data_en), list(raw_data_fr)

raw_data_en = [normalize_string(data) for data in raw_data_en]

raw_data_fr_in = ["<start> " + normalize_string(data) for data in raw_data_fr]
raw_data_fr_out = [normalize_string(data) + " <end>" for data in raw_data_fr]

In [7]:
raw_data_fr_in

['<start> Quel concept ridicule !',
 '<start> Votre idee n est pas completement folle.',
 '<start> La valeur d un homme reside dans ce qu il est.',
 '<start> Ce qu il a fait est tres mal.',
 '<start> Vous avez besoin de faire cela tous les trois.',
 '<start> Me donnez vous une autre chance ?',
 '<start> Tom et Mary travaillent tous les deux comme mannequins.',
 '<start> Puis je avoir quelques minutes je vous prie ?',
 '<start> Pourriez vous fermer la porte s il vous plait ?',
 '<start> Cette annee avez vous plante des citrouilles ?',
 '<start> Est ce que vous etudiez a la bibliotheque des fois ?',
 '<start> Ne vous laissez pas abuser par les apparences.',
 '<start> Je vous prie de m excuser ! Savez vous parler anglais ?',
 '<start> Peu de gens savent ce que cela veut reellement dire.',
 '<start> L Allemagne a produit beaucoup de scientifiques.',
 '<start> Devine de qui c est l anniversaire aujourd hui !',
 '<start> Il s est comporte comme s il possedait l endroit.',
 '<start> L honnete

In [6]:
raw_data_fr_out

['Quel concept ridicule ! <end>',
 'Votre idee n est pas completement folle. <end>',
 'La valeur d un homme reside dans ce qu il est. <end>',
 'Ce qu il a fait est tres mal. <end>',
 'Vous avez besoin de faire cela tous les trois. <end>',
 'Me donnez vous une autre chance ? <end>',
 'Tom et Mary travaillent tous les deux comme mannequins. <end>',
 'Puis je avoir quelques minutes je vous prie ? <end>',
 'Pourriez vous fermer la porte s il vous plait ? <end>',
 'Cette annee avez vous plante des citrouilles ? <end>',
 'Est ce que vous etudiez a la bibliotheque des fois ? <end>',
 'Ne vous laissez pas abuser par les apparences. <end>',
 'Je vous prie de m excuser ! Savez vous parler anglais ? <end>',
 'Peu de gens savent ce que cela veut reellement dire. <end>',
 'L Allemagne a produit beaucoup de scientifiques. <end>',
 'Devine de qui c est l anniversaire aujourd hui ! <end>',
 'Il s est comporte comme s il possedait l endroit. <end>',
 'L honnetete paye a la longue. <end>',
 'Comment sav